# Embeddings

Привет! Сегодня ты поработаешь с эмбеддингами: сделаешь классификатор эмоции твитов. Для начала, загрузи их:

In [1]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
84.9MB [00:00, 183MB/s]
Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Заимпортируй библиотеки и сделай работу скриптов вопсроизводимой.

In [2]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [3]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные

In [5]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Текст очень грязные. Надо добавить очистку текста в его предобработку. 

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [7]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Стокенизируем текст, избавим от знаков пунктуации и мелких слов.

In [8]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [9]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data . 

In [10]:
word2vec = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Эмбеддинги не нормализированны, поэтому это тоже надо делать (нейросети и не только любят нормальные данные).

In [12]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [13]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return filtered_line

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        embeddings = [(word2vec.get_vector(w) - mean) / std for w in tokens if w in word2vec and len(w) > 3]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [14]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

## Average embedding
---
Попробуем получить векторное представление предложения из эмбеддингов слов. Самый простой вариант: усреднить вектора по всем словам. Полученный вектор можно отправить любому классификатору как вектор признаков.

Посмотрим, насколько хорошо усреднее работает для определение эмоций твитов. Сделаем их визуализацию.

In [15]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Можно получить более аккуратными алгоритмами, но данный алгоритм покажет сложность задачи и поможет оценить требования к классификатору.

In [16]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2) 
pca.fit(examples["features"])
examples["transformed_features"] = pca.transform(examples["features"]) # Обучи PCA на эмбеддингах слов

In [17]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [18]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1002', ...)

Скорее всего, ты увидел не сильно различающиеся классы. Возможно, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую(хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [19]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Для обученния и тестирования нейросетевой модели сделаем отдельные функции.

In [20]:
from tqdm.notebook import tqdm

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)    
    return acc

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].float().to(device)
        targets = targets.unsqueeze(1)

        optimizer.zero_grad()
        
        y_pred = model(features)

        # Получи предсказания модели
        loss = criterion(y_pred, targets)# Посчитай лосс
        # Обнови параметры модели

        loss.backward()
        optimizer.step()


        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].float().to(device)
            targets = targets.unsqueeze(1)

            y_pred = model(features)# Получи предсказания модели

            loss = criterion(y_pred, targets) # Посчитай лосс
            
            acc = binary_acc(y_pred, targets)

            mean_loss += loss.item()  
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Можете создавать любую модель, с любым количеством слоев, функций активации и прочее.


In [21]:
import torch.nn as nn
from torch.optim import Adam

# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-2
num_epochs = 5
hl = 300

In [22]:
import torch.nn as nn
import torch.nn.functional as F
model = nn.Sequential(
    nn.Linear(vector_size, hl),
    nn.BatchNorm1d(hl),
    nn.ReLU(),
    nn.Linear(hl, hl),
    nn.BatchNorm1d(hl),
    nn.ReLU(),
    nn.Linear(hl, hl),
    nn.BatchNorm1d(hl),
    nn.ReLU(),
    nn.Linear(hl, hl),
    nn.BatchNorm1d(hl),
    nn.ReLU(),
    nn.Linear(hl, hl),
    nn.BatchNorm1d(hl),
    nn.ReLU(),
    nn.Linear(hl, hl),
    nn.BatchNorm1d(hl),
    nn.ReLU(),
    nn.Linear(hl, hl),
    nn.BatchNorm1d(hl),
    nn.ReLU(),
    nn.Linear(hl, num_classes)
)

model = model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

Обучим модель и протестируем её.
После каждой эпохи, проверим качество модели по валидационной части датасета. Если метрика стала лучше, будем сохранять модель. Подумай, какая метрика (точность или лосс) будет лучше работать в этой задаче? 

Здесь и далее реализованно с помощью лосс. Если думаешь, что лучше сравнивать модель через качество, то поменяй код выбора модели.

In [23]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5226051009893418, 'Test Acc': 73.82}



{'Test Loss': 0.5086781848669052, 'Test Acc': 74.74}



{'Test Loss': 0.5041814179420471, 'Test Acc': 74.9}



{'Test Loss': 0.5057652416229248, 'Test Acc': 75.0}



{'Test Loss': 0.5018710366487503, 'Test Acc': 75.172}


In [24]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.504327917060913, 'Test Acc': 74.98083067092652}


## TF-iDF
---

Вместо обычного усреднения эмбеддингов их можно дополнительно перевзвесить. Для этого воспользуемся алгоритмом `TD-iDF`. Он уже реализован в библиотеке `scikit-learn`, остается только его добавить в наш пайплайн.

In [83]:
from collections import defaultdict
from typing import Dict

from sklearn.feature_extraction.text import TfidfVectorizer


class TwitterDatasetTfIdf(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, weights: Dict[str, float] = None):
        super().__init__(data, feature_column, target_column, word2vec)

        if weights is None:
            self.weights = self.get_tf_idf_()
        else:
            self.weights = weights

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std  * self.weights.get(token, 1) for token in tokens if token in word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return filtered_line


    def get_tf_idf_(self):
        # Надо обучить tfidf на очищенном тексте. Но он принимает только список текстов, а не список списка токенов. Надо превратить второе в первое
        
        tokenized_texts = [' '.join(self.get_tokens_(row)) for row in self.data[self.feature_column]]

        tf_idf = TfidfVectorizer()
        tf_idf.fit_transform(tokenized_texts)
        # Обучи tf-idf
        return dict(zip(tf_idf.get_feature_names(), tf_idf.idf_))


In [84]:
dev = TwitterDatasetTfIdf(dev_data, "text", "emotion", word2vec)

Посмотрим на сложность получившейся задачи используя визуализацию через `PCA`.

In [85]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [86]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)

pca.fit(examples["features"])
examples["transformed_features"] = pca.transform(examples["features"])

In [87]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1223', ...)

Создать нейросетку, обучим её на этих данных.

In [88]:
train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [89]:
# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-2
num_epochs = 5
hl = 300

model = model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

In [90]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    print(testing(model, criterion, valid_loader, device))
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.4953661975860596, 'Test Acc': 75.432}



{'Test Loss': 0.4953661975860596, 'Test Acc': 75.432}



{'Test Loss': 0.49831942570209503, 'Test Acc': 75.144}



{'Test Loss': 0.49831942570209503, 'Test Acc': 75.144}



{'Test Loss': 0.499907766699791, 'Test Acc': 75.064}



{'Test Loss': 0.499907766699791, 'Test Acc': 75.064}



{'Test Loss': 0.504128697514534, 'Test Acc': 74.964}



{'Test Loss': 0.504128697514534, 'Test Acc': 74.964}



{'Test Loss': 0.5125136147737503, 'Test Acc': 74.652}



{'Test Loss': 0.5125136147737503, 'Test Acc': 74.652}


In [91]:
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec, weights=dev.weights)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5147479346956307, 'Test Acc': 74.18849840255591}


Есть ли разница в качестве между способами? Получилось ли улучшить качество модели?

----
<Качество толком не изменилось. Но этого стоило ожидать. Дело в том, что признаком слишком много. А метод PCA снижает размерность не так сильно, возможно стоило бы попробовать TSNE метод>

----

Сделай небольшое исследование:
- Попробуй сделать несколько нейросеток в качестве классификатора
- Попробуй другие предобученные эмбеддинги
- Попробуй очистить текст от ников ("@username"), url-ов и других символов

Для реализации последнего тебе могут помочь регулярные выражения (`import re`). Напише ниже отчет, что ты попробовал и что получилось.

---

<Попробовал ваш совет по обработке текстов, а именно удаление ссылок и юзернеймов. Как таковых результатов не дало. Также я пробовал реализовать задачу на RNN архитектуре, но мне кажется это не будет работать, так как связи между твитами не так много. Это больше поможет для обработки больших текстов, где нужна связь между предложениями. Почему же качество не могу поднять выше 76? Моё предположение - эмбеддинги. Дело в том, что я не могу положить более серьёзные эмбеддинги из-за ограничений ОЗУ. Мне кажется, очень хорошо для задачи классификации твитов поможет fasttext, т.к. там будут много слов, которых нет в словаре, а фаст текст хорошо с этим справляется :) P.S. изнначально изменил эмбеддинги, использовал вики300, telegram : logisticregression, пишите если что:)>

---

In [92]:
import re

In [93]:
class TwitterDatasetTfIdf_with_clear(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, weights: Dict[str, float] = None):
        super().__init__(data, feature_column, target_column, word2vec)

        if weights is None:
            self.weights = self.get_tf_idf_()
        else:
            self.weights = weights

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std  * self.weights.get(token, 1) for token in tokens if token in word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return filtered_line

    
    def clear_usernames_and_links_(self, text):
        text = re.sub(r'[@]\S*', '', text)
        text = re.sub(r'[http]\S*', '', text)
        return text


    def get_tf_idf_(self):
        # Надо обучить tfidf на очищенном тексте. Но он принимает только список текстов, а не список списка токенов. Надо превратить второе в первое
        
        tokenized_texts = [' '.join(self.get_tokens_(row)) for row in self.data[self.feature_column]]

        tokenized_texts = [self.clear_usernames_and_links_(t) for t in tokenized_texts]

        tf_idf = TfidfVectorizer()
        tf_idf.fit_transform(tokenized_texts)
        # Обучи tf-idf
        return dict(zip(tf_idf.get_feature_names(), tf_idf.idf_))


In [94]:
dev_clear = TwitterDatasetTfIdf_with_clear(dev_data, "text", "emotion", word2vec)

{'feature': array([[-5.46476984e+00,  4.61514854e+00, -6.31430054e+00,
         -5.88621950e+00,  8.39518726e-01,  4.13813353e+00,
         -4.59409028e-01,  5.35050297e+00,  2.94935441e+00,
         -2.36531353e+01,  5.59104776e+00, -6.53551579e+00,
         -7.98807800e-01, -4.10373402e+00,  2.06930995e+00,
          2.70994973e+00, -4.63738871e+00, -2.30496836e+00,
          2.10175395e+00, -3.23177958e+00, -5.13372302e-01,
          2.21646309e+00,  9.27336502e+00,  2.59989595e+00,
         -2.97559595e+00,  4.99345446e+00,  3.10563016e+00,
          1.27033448e+00, -2.17884135e+00,  4.73508596e-01,
          6.34210396e+00,  4.49472666e+00,  1.64247215e+00,
          2.50040746e+00, -1.94575710e+01,  2.72121406e+00,
          2.07739639e+00, -2.37691283e+00,  5.86487472e-01,
         -2.00421381e+00, -1.43217742e+00, -4.48968267e+00,
         -6.22430420e+00,  2.83584285e+00, -3.94346070e+00,
         -4.31319857e+00, -3.97112697e-01,  3.55439901e+00,
         -3.96924663e+00, -1.

In [95]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev_clear[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev_clear[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

from sklearn.decomposition import PCA


pca = PCA(n_components=2)

pca.fit(examples["features"])
examples["transformed_features"] = pca.transform(examples["features"])

draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

1280


Figure(id='1347', ...)

In [96]:
train_size = math.ceil(len(dev_clear) * 0.8)

train, valid = random_split(dev_clear, [train_size, len(dev_clear) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [98]:
batch_size
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-2
num_epochs = 10
hl = 32

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    print(testing(model, criterion, valid_loader, device))
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5302167975902558, 'Test Acc': 73.188}



{'Test Loss': 0.5302167975902558, 'Test Acc': 73.188}



{'Test Loss': 0.5303972668647766, 'Test Acc': 73.116}



{'Test Loss': 0.5303972668647766, 'Test Acc': 73.116}



{'Test Loss': 0.5325922399759293, 'Test Acc': 72.808}


...


{'Test Loss': 0.5385799173116684, 'Test Acc': 72.748}



{'Test Loss': 0.5407526606321335, 'Test Acc': 72.684}



{'Test Loss': 0.5407526606321335, 'Test Acc': 72.684}



{'Test Loss': 0.5467753150463104, 'Test Acc': 72.444}



{'Test Loss': 0.5467753150463104, 'Test Acc': 72.444}


In [100]:
test = TwitterDatasetTfIdf_with_clear(test_data, "text", "emotion", word2vec, weights=dev.weights)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5211084634541704, 'Test Acc': 73.6773162939297}


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

class RNN(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(RNN, self).__init__()

		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		"""

		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False)
		self.rnn = nn.RNN(embedding_length, hidden_size, num_layers=2, bidirectional=True)
		self.label = nn.Linear(4*hidden_size, output_size)
	
	def forward(self, input_sentences, batch_size=None):
		
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class which receives its input as the final_hidden_state of RNN.
		logits.size() = (batch_size, output_size)
		
		"""

		input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			h_0 = Variable(torch.zeros(4, self.batch_size, self.hidden_size).cuda()) # 4 = num_layers*num_directions
		else:
			h_0 =  Variable(torch.zeros(4, batch_size, self.hidden_size).cuda())
		output, h_n = self.rnn(input, h_0)
		# h_n.size() = (4, batch_size, hidden_size)
		h_n = h_n.permute(1, 0, 2) # h_n.size() = (batch_size, 4, hidden_size)
		h_n = h_n.contiguous().view(h_n.size()[0], h_n.size()[1]*h_n.size()[2])
		# h_n.size() = (batch_size, 4*hidden_size)
		logits = self.label(h_n) # logits.size() = (batch_size, output_size)
		
		return logits